# Harmonisation strategies
Now the we have seen how different scanners can produce difference in images between the same individual on different scanners, we need to figure out to account for this data. 

In [1]:
import os
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import pandas as pd
import datalad.api as dl

Let's grab the dataset that has all the imgaging volumes for the study and read it into memory

In [2]:
dl_source='/Users/davecash/Data/IDEAS/sample'
sample=dl.clone(dl_source,path='/tmp/sample',description='Cloned sample dataset for import')
in_file=sample.get('./GENFI_DEMON_SPREADSHEET.xlsx')

[INFO] Cloning dataset to Dataset(/tmp/sample) 
[INFO] Attempting to clone from /Users/davecash/Data/IDEAS/sample to /tmp/sample 
[INFO] Completed clone attempts for Dataset(/tmp/sample) 


In [3]:
df_xsec=pd.read_excel(in_file[0]['path'])

A quick cross tab to show the different kinds of scanners used at each site.

In [4]:
pd.crosstab(df_xsec['Site'],df_xsec['Scanner'])

Scanner,GE 1.5T,GE 3T,Philips 3T,Siemens 1.5T,Siemens Prisma 3T,Siemens Skyra 3T,Siemens Trio 3T
Site,,,,,,,
0,0,0,0,0,0,8,20
1,0,0,45,0,0,0,0
2,0,0,0,0,3,0,0
3,0,0,143,0,0,0,0
4,0,0,3,0,0,0,0
5,0,25,0,0,6,0,0
6,0,0,18,0,0,0,0
7,0,0,0,0,10,0,12
8,0,0,0,33,0,36,0


## Analysis
Now that we have the data,we have three groups. Let's take a look at a couple of structures and see what edifferences they are.
### Simple regression no sites

## Checking for site differences
Are their differences in site or scanner that are occuring in this data. Let's run the same analysis again including them as a covariate.

This is really helpful, but what if there are difference in standard error between sites, and how can we trust those small sample size in some cells. Let's try this with COMbat instead.